<h1>  Importing Libraries   <h1>

In [1]:
import pandas as pd
import psycopg2
import json

<h1>   Importing JSON file  <h1>

In [2]:
with open("/home/ys/notebook/config.json") as f:
    config = json.load(f)
print("Config:",config)
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')

Config: {'database': 'Students', 'user': 'postgres', 'password': 'a', 'host': 'localhost', 'port': '5432'}


<h1> Professor tble columns    <h1>

In [3]:
mycursor = mydb.cursor()
table="professor"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'Students_db' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
     print(value[3],":",value[7])

id : integer
first_name : character varying
last_name : character varying
email : character varying


<h1>  Adding professor function   <h1>

In [4]:
def add_professor(prenom,nom,email):
    with open("/home/ys/notebook/config.json") as f:
        config = json.load(f)
    
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"""SELECT * FROM "Students_db".professor where email ='{email}';""")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = """INSERT INTO "Students_db".professor (first_name,last_name,email) VALUES (%s,%s,%s)"""
        val = (prenom,nom,email)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"""SELECT * FROM "Students_db".professor where email ='{email}';""")
        myresult = mycursor.fetchall()
        return myresult[0][0]

<h1>   Adding Professor to table  <h1>

In [5]:
df = pd.read_csv("/home/ys/notebook/processed/prof_list_processed.csv")
for index, row in df.iterrows():
    nom=row['last_name']
    prenom=row['first_name']
    email=row['email']

    id_professor=add_professor(prenom,nom,email)
    print(id_professor)

1
